# ペプチド分子量計算

アミノ酸配列からペプチドの分子量（Molecular Weight）を計算するノートブック。

## 機能
- モノアイソトピック質量 / 平均質量の計算
- m/z値の計算（質量分析用）
- アミノ酸組成の解析
- 翻訳後修飾（PTM）を考慮した質量計算
- FASTAファイルからの一括計算

In [ ]:
import pandas as pd
from peptide_mw import (
    calculate_mw,
    calculate_mz,
    amino_acid_composition,
    sequence_summary,
    MODIFICATIONS,
)

## 1. 基本的な分子量計算

In [ ]:
# 単一ペプチドの分子量計算
sequence = "SIINFEKL"  # OVA257-264 (マウスMHC-Iエピトープ)

mw_mono = calculate_mw(sequence, mass_type="monoisotopic")
mw_avg = calculate_mw(sequence, mass_type="average")

print(f"配列: {sequence}")
print(f"モノアイソトピック質量: {mw_mono:.5f} Da")
print(f"平均質量:             {mw_avg:.5f} Da")

## 2. 配列の概要情報

In [ ]:
# 複数のネオアンチゲン候補ペプチドを解析
peptides = [
    "SIINFEKL",      # OVA257-264
    "GILGFVFTL",     # Influenza M1 (HLA-A*02:01)
    "NLVPMVATV",     # CMV pp65 (HLA-A*02:01)
    "GLCTLVAML",     # EBV BMLF1 (HLA-A*02:01)
    "YLQPRTFLL",     # SARS-CoV-2 Spike (HLA-A*02:01)
]

results = []
for pep in peptides:
    info = sequence_summary(pep)
    results.append({
        "Sequence": info["sequence"],
        "Length": info["length"],
        "MW (Da)": info["molecular_weight"],
        "[M+H]+": info["mz_1"],
        "[M+2H]2+": info["mz_2"],
        "[M+3H]3+": info["mz_3"],
    })

df = pd.DataFrame(results)
df

## 3. 翻訳後修飾（PTM）を考慮した計算

In [ ]:
# 利用可能な修飾一覧
print("利用可能な修飾:")
for name, mass in MODIFICATIONS.items():
    print(f"  {name}: +{mass:.5f} Da")

In [ ]:
# 修飾付きペプチドの分子量計算例
sequence = "PEPTMIDE"

# 修飾なし
mw_unmod = calculate_mw(sequence)
print(f"配列: {sequence}")
print(f"修飾なし: {mw_unmod:.5f} Da")

# メチオニン酸化 (位置4のM)
mw_ox = calculate_mw(sequence, modifications=[{"position": 4, "type": "oxidation"}])
print(f"Met酸化:  {mw_ox:.5f} Da (差: +{mw_ox - mw_unmod:.5f})")

# リン酸化 + 酸化
mw_multi = calculate_mw(sequence, modifications=[
    {"position": 4, "type": "oxidation"},
    {"position": 0, "type": "phosphorylation"},  # N末端近傍のリン酸化
])
print(f"リン酸化+酸化: {mw_multi:.5f} Da (差: +{mw_multi - mw_unmod:.5f})")

## 4. アミノ酸組成の解析

In [ ]:
sequence = "GILGFVFTL"
comp = amino_acid_composition(sequence)

print(f"配列: {sequence}")
print(f"長さ: {len(sequence)} aa")
print(f"\nアミノ酸組成:")
for aa, count in comp.items():
    print(f"  {aa}: {count} ({count/len(sequence)*100:.1f}%)")

## 5. m/z値の計算（質量分析用）

In [ ]:
sequence = "YLQPRTFLL"
print(f"配列: {sequence}")
print(f"MW: {calculate_mw(sequence):.5f} Da\n")

# 各荷電状態のm/z
for z in range(1, 5):
    mz = calculate_mz(sequence, charge=z)
    print(f"  [M+{z}H]{z}+: {mz:.5f}")

## 6. FASTAファイルからの一括計算

In [ ]:
def parse_fasta(filepath, max_sequences=None):
    """FASTAファイルを読み込み、(header, sequence) のリストを返す。"""
    sequences = []
    header = None
    seq_parts = []

    with open(filepath) as f:
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if header is not None:
                    sequences.append((header, "".join(seq_parts)))
                    if max_sequences and len(sequences) >= max_sequences:
                        return sequences
                header = line[1:]
                seq_parts = []
            else:
                seq_parts.append(line)
        if header is not None:
            sequences.append((header, "".join(seq_parts)))

    return sequences


# deeploc_data.fastaから最初の10配列を読み込み
fasta_path = "20250216/data/deeploc_data.fasta"
sequences = parse_fasta(fasta_path, max_sequences=10)

results = []
for header, seq in sequences:
    try:
        mw = calculate_mw(seq, mass_type="average")
        results.append({
            "Header": header[:50],
            "Length": len(seq),
            "MW (Da)": round(mw, 2),
            "MW (kDa)": round(mw / 1000, 2),
        })
    except ValueError as e:
        results.append({
            "Header": header[:50],
            "Length": len(seq),
            "MW (Da)": f"Error: {e}",
            "MW (kDa)": "-",
        })

df_fasta = pd.DataFrame(results)
df_fasta

## 7. カスタム配列の計算

以下のセルに解析したいペプチド配列を入力してください。

In [ ]:
# ここに解析したい配列を入力
my_sequence = "ACDEFGHIKLMNPQRSTVWY"

info = sequence_summary(my_sequence)
print(f"配列:       {info['sequence']}")
print(f"長さ:       {info['length']} aa")
print(f"分子量:     {info['molecular_weight']:.5f} Da")
print(f"[M+H]+:     {info['mz_1']:.5f}")
print(f"[M+2H]2+:   {info['mz_2']:.5f}")
print(f"[M+3H]3+:   {info['mz_3']:.5f}")
print(f"\nアミノ酸組成:")
for aa, count in info['composition'].items():
    print(f"  {aa}: {count}")